# Custom State with Memory - LangChain 1.0

Advanced pattern: Custom agent state that persists across conversation.

**Features:**
- Custom state schema with employee data
- State persists across conversation
- Persistent conversation memory
- Production-ready pattern
- Gradio interface

In [5]:
# Install dependencies
!pip install --pre -U langchain langchain-openai langgraph gradio

In [6]:
from google.colab import userdata
import os

OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [7]:
from langchain_core.tools import tool
from typing import Annotated

# Employee database
EMPLOYEE_DB = {
    "101": {"name": "Priya Sharma", "department": "Engineering", "leave": 12},
    "102": {"name": "Rahul Verma", "department": "Engineering", "leave": 8},
    "103": {"name": "Anjali Patel", "department": "HR", "leave": 15},
}

# Global state for current session (in production, use proper state management)
current_session = {
    "employee_id": None,
    "employee_name": None,
    "department": None,
    "leave_balance": 0
}

# Tools that work with session state
@tool
def set_employee_context(employee_id: Annotated[str, "Employee ID"]) -> str:
    """Set the current employee context for the session."""
    if employee_id in EMPLOYEE_DB:
        emp = EMPLOYEE_DB[employee_id]
        current_session["employee_id"] = employee_id
        current_session["employee_name"] = emp["name"]
        current_session["department"] = emp["department"]
        current_session["leave_balance"] = emp["leave"]
        return f"✅ Context set for {emp['name']} (ID: {employee_id}, Department: {emp['department']})"
    return f"❌ Employee ID {employee_id} not found."

@tool
def get_my_info() -> str:
    """Get information about the current employee."""
    if not current_session["employee_id"]:
        return "⚠️ No employee context set. Please identify yourself first."

    return f"""Your Information:
📋 Name: {current_session['employee_name']}
🆔 ID: {current_session['employee_id']}
🏢 Department: {current_session['department']}
🌴 Leave Balance: {current_session['leave_balance']} days"""

@tool
def check_leave_balance() -> str:
    """Check leave balance for current employee."""
    if not current_session["employee_id"]:
        return "⚠️ No employee context set. Please identify yourself first."

    name = current_session['employee_name']
    leave = current_session['leave_balance']
    return f"{name}, you have {leave} days of leave available."

@tool
def request_leave(days: Annotated[int, "Number of days"]) -> str:
    """Request leave days. Checks against available balance."""
    if not current_session["employee_id"]:
        return "⚠️ No employee context set. Please identify yourself first."

    name = current_session['employee_name']
    current_leave = current_session['leave_balance']

    if days > current_leave:
        return f"❌ Sorry {name}, you only have {current_leave} days available. Cannot approve {days} days."

    return f"✅ Leave request submitted for {name}: {days} days. Pending manager approval."

print("✅ Tools created!")

✅ Tools created!


In [9]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver

# Create agent with memory
employee_agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=[set_employee_context, get_my_info, check_leave_balance, request_leave],
    checkpointer=InMemorySaver(),
    system_prompt="""You are a helpful HR assistant.

    When an employee introduces themselves or mentions their employee ID,
    use the set_employee_context tool to set up their session.

    After that, you can use other tools to help them without needing their ID again.

    Be professional and helpful."""
)

print("✅ Employee agent created!")

✅ Employee agent created!


In [10]:
# Test the agent
config = {"configurable": {"thread_id": "employee_session_1"}}

print("=" * 70)
print("Testing Employee Agent with Context")
print("=" * 70 + "\n")

# Turn 1: Employee introduces themselves
result = employee_agent.invoke(
    {"messages": [{"role": "user", "content": "Hi, I'm employee 101, Priya Sharma"}]},
    config
)
print("User: Hi, I'm employee 101, Priya Sharma")
print(f"Agent: {result['messages'][-1].content}\n")

# Turn 2: Ask about leave
result = employee_agent.invoke(
    {"messages": [{"role": "user", "content": "How much leave do I have?"}]},
    config
)
print("User: How much leave do I have?")
print(f"Agent: {result['messages'][-1].content}\n")

# Turn 3: Apply for leave
result = employee_agent.invoke(
    {"messages": [{"role": "user", "content": "I want to apply for 5 days of leave"}]},
    config
)
print("User: I want to apply for 5 days of leave")
print(f"Agent: {result['messages'][-1].content}\n")

# Turn 4: Check info
result = employee_agent.invoke(
    {"messages": [{"role": "user", "content": "Show me my full information"}]},
    config
)
print("User: Show me my full information")
print(f"Agent: {result['messages'][-1].content}")

print("\n✅ Agent maintained context throughout conversation!")

Testing Employee Agent with Context

User: Hi, I'm employee 101, Priya Sharma
Agent: Hello Priya Sharma! How can I assist you today?

User: How much leave do I have?
Agent: You have 12 days of leave available. If you need assistance with anything else, feel free to ask!

User: I want to apply for 5 days of leave
Agent: Your leave request for 5 days has been submitted and is pending manager approval. If you need further assistance or have any questions, feel free to let me know!

User: Show me my full information
Agent: Here is your full information:

- **Name:** Priya Sharma
- **Employee ID:** 101
- **Department:** Engineering
- **Leave Balance:** 12 days

If there’s anything else you need, just let me know!

✅ Agent maintained context throughout conversation!


In [11]:
import gradio as gr
import uuid

# Session management
session_id = str(uuid.uuid4())

def reset_session():
    """Reset the current session."""
    global current_session
    current_session = {
        "employee_id": None,
        "employee_name": None,
        "department": None,
        "leave_balance": 0
    }
    return "Session reset. Please introduce yourself again."

def chat_with_context(message, history):
    """Chat with employee context."""
    try:
        config = {"configurable": {"thread_id": session_id}}

        result = employee_agent.invoke(
            {"messages": [{"role": "user", "content": message}]},
            config
        )
        return result['messages'][-1].content
    except Exception as e:
        return f"Error: {str(e)}"

# Create Gradio interface
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 👤 Employee Portal with Context - LangChain 1.0")
    gr.Markdown("""This demonstrates context management with memory.

    **How it works:**
    1. Introduce yourself with your employee ID (101, 102, or 103)
    2. Agent sets your context automatically
    3. All future queries use this context
    4. No need to repeat your ID!

    **Try it:**
    - "Hi, I'm employee 101"
    - "How much leave do I have?"
    - "Show me my information"
    """)

    with gr.Row():
        reset_btn = gr.Button("Reset Session", variant="secondary")

    reset_output = gr.Textbox(label="Status", interactive=False)

    chatbot = gr.ChatInterface(
        fn=chat_with_context,
        examples=[
            "Hi, I'm employee 101, Priya Sharma",
            "How much leave do I have?",
            "I want to request 3 days of leave",
            "Show me my full information",
            "What department am I in?"
        ],
    )

    reset_btn.click(reset_session, outputs=[reset_output])

    gr.Markdown("### 📋 Available Employees")
    employee_list = "\n".join([f"- **{emp_id}**: {emp['name']} ({emp['department']}, {emp['leave']} days leave)"
                               for emp_id, emp in EMPLOYEE_DB.items()])
    gr.Markdown(employee_list)

print(f"Session ID: {session_id}")
print("\nAvailable employees:")
for emp_id, emp in EMPLOYEE_DB.items():
    print(f"  - {emp_id}: {emp['name']} ({emp['department']}, {emp['leave']} days leave)")

# Launch the interface
demo.launch()

Session ID: e77893e1-1c09-49ee-aace-945685ec8baf

Available employees:
  - 101: Priya Sharma (Engineering, 12 days leave)
  - 102: Rahul Verma (Engineering, 8 days leave)
  - 103: Anjali Patel (HR, 15 days leave)
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://072a2ad20ee59d5751.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## How This Pattern Works

### 1. Session Context Management
Instead of using complex state injection, we use a simple session dictionary:
```python
current_session = {
    "employee_id": None,
    "employee_name": None,
    "department": None,
    "leave_balance": 0
}
```

### 2. Context Setting Tool
```python
@tool
def set_employee_context(employee_id: str):
    # Load employee data
    # Update session context
    current_session["employee_id"] = employee_id
    # ...
```

### 3. Context-Aware Tools
All other tools check the session context:
```python
@tool
def check_leave_balance():
    if not current_session["employee_id"]:
        return "Please identify yourself first"
    # Use session data
    return f"You have {current_session['leave_balance']} days"
```

### 4. Conversation Memory
LangChain's checkpointer handles conversation history:
```python
agent = create_agent(
    checkpointer=InMemorySaver(),  # Remembers conversation
    ...
)
```

## Benefits of This Approach

✅ **Simple** - No complex state injection  
✅ **Works** - Compatible with current LangChain  
✅ **Clear** - Easy to understand and debug  
✅ **Flexible** - Easy to extend with more context  

## Production Considerations

### For Multi-User Production Systems

Replace the global `current_session` with:

**Option 1: Thread-based storage**
```python
session_store = {}  # {thread_id: session_data}

@tool
def get_my_info():
    thread_id = get_current_thread_id()  # From context
    session = session_store.get(thread_id, {})
    return session.get("employee_name")
```

**Option 2: Database storage**
```python
@tool
def get_my_info():
    thread_id = get_current_thread_id()
    session = db.get_session(thread_id)
    return session.employee_name
```

**Option 3: Redis for distributed systems**
```python
@tool
def get_my_info():
    thread_id = get_current_thread_id()
    session = redis.get(f"session:{thread_id}")
    return json.loads(session)["employee_name"]
```

## Example Conversation Flow

```
User: Hi, I'm employee 102
Agent: [Calls set_employee_context("102")]
      ✅ Context set for Rahul Verma
      Hello Rahul! How can I help you today?

User: How much leave do I have?
Agent: [Calls check_leave_balance()]
      [Tool reads from current_session]
      Rahul Verma, you have 8 days of leave available.

User: I want 3 days off
Agent: [Calls request_leave(3)]
      [Tool reads from current_session]
      ✅ Leave request submitted for Rahul Verma: 3 days
```

## Why This Works Better

1. **No Import Errors** - Uses standard Python patterns
2. **Easy Testing** - Simple to test and debug
3. **Maintainable** - Clear code structure
4. **Upgradable** - Easy to switch to database storage

## Next Steps

- Implement thread-safe session storage
- Add session expiration
- Store in database for persistence
- Add authentication
- Implement session cleanup